In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


running k-means on cuda:0..


[running kmeans]: 7it [00:00, 37.70it/s, center_shift=0.000030, iteration=7, tol=0.000100]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=2)
decomp.load()

Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.3.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.4.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from cache


In [3]:
decomp.lattice_scores[0].shape

(50, 50)

In [4]:
from typing import List
import utils
importlib.reload(utils)
modified_lattice = decomp.lattice_scores.copy()

# TODO: have function which does something like fixing from the a specific neuron?
# Ie have the sub-lattice with only **related** neurons

def cutoff_lattice(lattice: List[List[List[float]]], related_cutoff = 1):
    print(lattice[0].sum())
    r = [(layer > related_cutoff) * layer for layer in lattice]
    print(r[0].sum())
    return r

def create_sublattice(lattice, layer: int, idx_in_layer: int, related_cutoff = 10):
	with_cutoff = cutoff_lattice(lattice, related_cutoff)

LAST_FEAT = 20
modified_lattice = cutoff_lattice(decomp.lattice_scores, related_cutoff=15)

# utils.top_k_dag_paths_dynamic(modified_lattice, k=10_000)

27547.99987654594
1111.8467246356354


In [24]:
# TODO:
""""
We need to combine the 
"""

import numpy as np

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
# score_path = [39, 14, 2, 3, 4]
score_path = [1, 1, 2, 3, 4]
"""
Paths which seem to have meaning

score_path = [1, 1, 2, 3, 4]
Very strong for Q: . Specifically on the space
"""

to_score = [d[:100] for d in ds][:300]
scores = decomp.score(
    to_score,
    score_path=score_path,
    weighting_per_layer=np.array([1, 1, 1, 1, 1])
)


DS (5, 8250, 512) 8250
(8250, 5, 512) [1, 1, 2, 3, 4]
PROBS MAX 0.9999813 PROBS PATH 3.750224e-07
PROBS MAX 0.9999725 PROBS PATH 5.4569955e-07
PROBS MAX 0.31997374 PROBS PATH 0.013836986
PROBS MAX 0.32280213 PROBS PATH 0.013594408
PROBS MAX 0.21167986 PROBS PATH 0.0160584
PROBS MAX 0.028271277 PROBS PATH 0.021095825
PROBS MAX 0.029634718 PROBS PATH 0.019903755
PROBS MAX 0.030509083 PROBS PATH 0.019771356
PROBS MAX 0.027176943 PROBS PATH 0.021342833
PROBS MAX 0.02351144 PROBS PATH 0.019302985
PROBS MAX 0.02496543 PROBS PATH 0.019891737
PROBS MAX 0.025786312 PROBS PATH 0.02000212
PROBS MAX 0.025962278 PROBS PATH 0.01980698
PROBS MAX 0.025687654 PROBS PATH 0.025687654
PROBS MAX 0.02350304 PROBS PATH 0.021240305
PROBS MAX 0.02393093 PROBS PATH 0.019530533
PROBS MAX 0.02331908 PROBS PATH 0.020867374
PROBS MAX 0.024811622 PROBS PATH 0.020009834
PROBS MAX 0.025732007 PROBS PATH 0.023661617
PROBS MAX 0.025655447 PROBS PATH 0.022355346
PROBS MAX 0.03628328 PROBS PATH 0.01693539
PROBS MAX 0.0299

In [25]:
scores[10]

[0.04349071496881152,
 0.10600323416292667,
 0.10539226047694683,
 0.1039603129029274,
 0.1032944768667221,
 0.10511723347008228,
 0.10126306116580963,
 0.10584714263677597,
 0.10834402590990067,
 0.10626458376646042,
 0.09907666966319084,
 0.1055152453482151,
 0.10596247017383575,
 0.10768009722232819,
 0.10418650507926941,
 0.1051515992730856,
 0.10494630225002766,
 0.10609700158238411,
 0.09995151311159134,
 0.10507425665855408,
 0.1019435916095972,
 0.10112181678414345,
 0.10360377840697765,
 0.10441664047539234,
 0.10598589107394218,
 0.1081016045063734,
 0.10474619269371033,
 0.10789849050343037,
 0.1063727680593729]

In [26]:
import math
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!
act_simp = scores_reord
act_simp = [[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 100))]] for tok in s] for s in scores_reord]
# TODO sep fun
from circuitsvis.utils.render import RenderedHTML, render
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)

In [27]:
scores_per_token_set[top_args]

array([0.12130724, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12101022, 0.12101022,
       0.12101022, 0.12101022, 0.12101022, 0.12095588, 0.12068748,
       0.12003792, 0.11997999, 0.11989397, 0.11917453, 0.1189322 ,
       0.11891924, 0.11818246, 0.11818246, 0.11818246, 0.11818246,
       0.11818246, 0.11780145, 0.11719943, 0.11632757, 0.11618495,
       0.11565934, 0.11562371, 0.1154728 , 0.11521844, 0.11502

In [28]:
print(html)

<div id="circuits-vis-d7cc2241-467d" style="margin: 15px 0;"/>
    <script crossorigin type="module">
    import { render, TextNeuronActivations } from "https://unpkg.com/circuitsvis@1.43.2/dist/cdn/esm.js";
    render(
      "circuits-vis-d7cc2241-467d",
      TextNeuronActivations,
      {"tokens": [["[", "In", "n", "ovation", " in", " the", " incorporation", " of", " mac", "ron", "ut", "rients", " to", " ent", "eral", " nutrition", " formulas", "].", "\n", "Enter", "al", " nutrition"], ["Q", ":", "\n", "\n", "Res", "h", "aping", " a", " tensor", " with", " more", " than", " one", " u", "known", " di", "em", "ns", "ions", "\n", "\n", "I", " need", " to", " be", " able", " to", " resh", "ape", " a", " tensor", " o"], ["Q", ":", "\n", "\n", "error", " rg", "ding", " definition", " &", " no", " extension", " method", " for", " System", ".", "Web", ".", "R", "outing", ".", "Route", "Value", "Dictionary", "\n", "\n", "I", " am", " "], ["Q", ":", "\n", "\n", "Creating", " form", " with", "

In [29]:
from IPython.core.display import display, HTML

display(HTML(str(html)))

/tmp/ipykernel_418165/288767411.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Isolate Specific Neurons